In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

Load Behavioural Model

In [11]:
# === Behavioural Model ===
class Behavioural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)
behavior_pi = Behavioural(input_dim=5, hidden_dim=64, output_dim=2)

# 2. Load the weights into it
behavior_pi.load_state_dict(torch.load("./Behavioural_model.pth"))

<All keys matched successfully>

Pertubation  $\theta = \theta' (1 + \epsilon), \epsilon \backsim N(0,\sigma^2)$

In [ ]:
def get_multiplicatively_perturbed_copy(pi, sigma=0.01):
    # Clone the network
    pi_perturbed = type(pi)()
    pi_perturbed.load_state_dict(pi.state_dict())  # deep copy of weights

    # Apply multiplicative Gaussian noise: θ ← θ * (1 + ε)
    with torch.no_grad():
        for param in pi_perturbed.parameters():
            if param.requires_grad:
                noise = torch.randn_like(param) * sigma
                param.mul_(1 + noise)

    return pi_perturbed

perturb_size = 100
pi_list = []
for i in range(perturb_size):
    pi_perturbed = get_multiplicatively_perturbed_copy(behavior_pi, sigma=0.02)
    pi_list.append(pi_perturbed)


KL Divergence